In [1]:
import torch
import json

In [2]:
models = __import__("midas-api.ttc_classifier")
models = __import__("midas-api.ttp_regressor")
models = __import__("midas-api.atp_regressor")

#### showcase of 3 models:
1. title to category classifier
2. title to price regressor
3. attribute to price regressor (cars & motorcycles)

In [3]:
ttc = models.ttc_classifier
ttp = models.ttp_regressor
atp = models.atp_regressor

In [4]:
#ttc
cate_map_ix_obj = open('midas-api/temp/ttc_classifier_mapping.jsonl')
ttc_categories = ttc.Categories(cate_map_ix_obj)
ttc_model = ttc.Model(len(ttc_categories.ix_to_cate))
ttc_model.load_state_dict(torch.load('midas-api/temp/ttc_classifier.state', map_location='cpu'), strict=True)
ttc_model.eval()

#ttp
ttp_model = ttp.Model()
ttp_model.load_state_dict(torch.load('midas-api/temp/ttp_regressor.state', map_location='cpu'), strict=True)
ttp_model.eval()

#atp
attr_to_ix = json.load(open('midas-api/temp/atp_regressor.json'))
atp_model = atp.Model(len(attr_to_ix))
atp_model.load_state_dict(torch.load('midas-api/temp/atp_regressor.state', map_location='cpu'), strict=True)
atp_model.eval()
print('load models...')


load models...


In [5]:
ad_title = "Brand new iphone XR 64GB yellow"

In [6]:
ttc_input = ttc.make_input([ttc.preprocess_text(ad_title)])
cate_preds = ttc_model.forward(ttc_input).squeeze(0)
_, topk = cate_preds.topk(3)
for i, ix in enumerate(topk.tolist()):
    cate_ids_pred = ttc_categories.ix_to_cate[ix]
    print('rank',i + 1,       
        'cate_id', cate_ids_pred, ':',
        ttc_categories.cate_to_desc[cate_ids_pred])

rank 1 cate_id 89 : mobilephone
rank 2 cate_id 264 : mobile_cases
rank 3 cate_id 269 : mobile_accessories


In [7]:
ix_cate = ttp.categories.cate_to_ix[89] #mobilephone
ttp_input = ttp.make_input([ttp.preprocess_text(ad_title)], [ix_cate])
for r, price in zip(["Low", "Mid", "High"], ttp_model.forward(*ttp_input).squeeze().exp().type(torch.int32).tolist()):
    print(f'Suggested {r}: {price}', )


Suggested Low: 9904
Suggested Mid: 11182
Suggested High: 12881


In [8]:
ix_cate = ttp.categories.cate_to_ix[269] #mobile_accessories
ttp_input = ttp.make_input([ttp.preprocess_text(ad_title)], [ix_cate])
for r, price in zip(["Low", "Mid", "High"], ttp_model.forward(*ttp_input).squeeze().exp().type(torch.int32).tolist()):
    print(f'Suggested {r}: {price}', )


Suggested Low: 1680
Suggested Mid: 2272
Suggested High: 3806


In [9]:
sample_car1 = {'brand':'toyota', 'model':'vios', 'transmission': 'เกียร์อัตโนมัติ', 'year':'2018', 'cate_id': 11}
sample_car2 = {'brand':'mercedes-benz', 'model':'c-class', 'transmission': 'เกียร์อัตโนมัติ', 'year':'2019', 'cate_id': 11}

In [10]:
print(sample_car1)
inp_car = atp.get_attr_indices(sample_car1, attr_to_ix)
prices, score =  atp_model.predict(torch.tensor([inp_car]))
prices = prices.squeeze().exp().type(torch.int32).tolist()
for r, price in zip(["Low", "Mid", "High"], prices):
    print(f'Suggested {r}: {price}', )
# confidence = 1 - score.squeeze().item()

{'brand': 'toyota', 'model': 'vios', 'transmission': 'เกียร์อัตโนมัติ', 'year': '2018', 'cate_id': 11}
Suggested Low: 430693
Suggested Mid: 453632
Suggested High: 474619


In [11]:
print(sample_car2)
inp_car = atp.get_attr_indices(sample_car2, attr_to_ix)
prices, score =  atp_model.predict(torch.tensor([inp_car]))
prices = prices.squeeze().exp().type(torch.int32).tolist()
for r, price in zip(["Low", "Mid", "High"], prices):
    print(f'Suggested {r}: {price}', )
# confidence = 1 - score.squeeze().item()

{'brand': 'mercedes-benz', 'model': 'c-class', 'transmission': 'เกียร์อัตโนมัติ', 'year': '2019', 'cate_id': 11}
Suggested Low: 2031752
Suggested Mid: 2138648
Suggested High: 2181282
